In [1]:
import os
cwd = os.getcwd()
jaxqualin_path = os.path.join(cwd, os.pardir, os.pardir)

In [2]:
import sys
sys.path.insert(0, jaxqualin_path)

In [3]:
import jaxqualin.postprocess
import jaxqualin.plot
import jaxqualin.qnmode

from jaxqualin.postprocess import *
from jaxqualin.plot import *
from jaxqualin.qnmode import *

2023-10-01 20:51:46.264857: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2023-10-01 20:51:46.422729: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2023-10-01 20:51:46.435129: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIB

In [4]:
from importlib import reload

In [5]:
reload(jaxqualin.postprocess)
from jaxqualin.postprocess import *
reload(jaxqualin.plot)
from jaxqualin.plot import *
reload(jaxqualin.qnmode) 
from jaxqualin.qnmode import *

In [70]:
# from pmlb import fetch_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np

from scipy.interpolate import LinearNDInterpolator


In [7]:
import sys
import os

In [8]:
os.environ["PATH"] = "/home/mcheung1/julia-1.9.0/bin"+ os.pathsep + os.environ["PATH"]

In [9]:
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [10]:
settingname = "normal2"

ROOT_PATH = jaxqualin_path
PLOT_PATH = os.path.join(ROOT_PATH, f"plots/{settingname}/")
PLOT_PATH_ROOT = os.path.join(ROOT_PATH, f"plots/")

In [11]:
runname = "normal2_prodB1"
runname_validation = "normal2_prodB3"

df = pd.read_csv(f"../../pickle/data_frame/{runname}_physical.csv")
df = classify_modes(df)
df = df_change_retro_to_pro(df)

In [12]:
df_validation = pd.read_csv(f"../../pickle/data_frame/{runname_validation}_physical.csv")
df_validation = classify_modes(df_validation)
df_validation = df_change_retro_to_pro(df_validation)

In [13]:
df_full = pd.concat([df, df_validation])

In [14]:
def adjust_phi(row, df):
    phi_med = row['phi_med']
    SXS_num = row['SXS_num']
    retro = row['retro']
    mode_string = row['mode_string']
    if mode_string == 'constant':
        return phi_med
    m = int(mode_string.split('.')[1])
    row_220 = df[(df['SXS_num'] == SXS_num)&(df['l'] == 2)&(df['m'] == 2)&(df['mode_string'] == '2.2.0')]
    phi_220 = row_220['phi_med'].values[0]
    retro_fac = -1 if retro else 1
    return 2*phi_med - phi_220*np.abs(m)

In [15]:
phi_med_adj_col_full = df_full.apply(lambda row: adjust_phi(row, df_full), axis=1)
df_full['phi_med_adj'] = phi_med_adj_col_full % (2*np.pi)

In [16]:
phi_med_adj_col = df.apply(lambda row: adjust_phi(row, df), axis=1)
df['phi_med_adj'] = phi_med_adj_col % (2*np.pi)

In [17]:
# PN_dom_func = {
#     (2,2): lambda eta: eta,
#     (2,1): lambda eta: eta*np.sqrt(1 - 4*eta),
#     (3,3): lambda eta: eta*np.sqrt(1 - 4*eta),
#     (3,2): lambda eta: eta*(1 - 3*eta),
#     (3,1): lambda eta: eta*np.sqrt(1 - 4*eta),
#     (4,4): lambda eta: eta*(1 - 3*eta),
#     (4,3): lambda eta: eta*np.sqrt(1 - 4*eta)*(1 - 2*eta),
#     (4,2): lambda eta: eta*(1 - 3*eta),
#     (4,1): lambda eta: eta*np.sqrt(1 - 4*eta)*(1 - 2*eta),
#     (5,5): lambda eta: eta*np.sqrt(1 - 4*eta)*(1 - 2*eta),
#     (6,6): lambda eta: eta*(1 - 5*eta + 5*eta**2),
#     (7,7): lambda eta: eta*np.sqrt(1 - 4*eta)*(1 - 4*eta + 3*eta**2),
#     (8,8): lambda eta: eta*(1 - 7*eta + 14*eta**2 - 7*eta**3),
# }

# dom_func_tex_string = {
#     (2,2): r"$\eta$",
#     (2,1): r"$\eta\sqrt{1 - 4\eta}$",
#     (3,3): r"$\eta\sqrt{1 - 4\eta}$",
#     (3,2): r"$\eta(1 - 3\eta)$",
#     (3,1): r"$\eta\sqrt{1 - 4\eta}$",
#     (4,4): r"$\eta(1 - 3\eta)$",
#     (4,3): r"$\eta\sqrt{1 - 4\eta}(1 - 2\eta)$",
#     (4,2): r"$\eta(1 - 3\eta)$",
#     (4,1): r"$\eta\sqrt{1 - 4\eta}(1 - 2\eta)$",
#     (5,5): r"$\eta\sqrt{1 - 4\eta}(1 - 2\eta)$",
#     (6,6): r"$\eta(1 - 5\eta + 5\eta^2)$",
#     (7,7): r"$\eta\sqrt{1 - 4\eta}(1 - 4\eta + 3\eta^2)$",
#     (8,8): r"$\eta(1 - 7\eta + 14\eta^2 - 7\eta^3)$",
# }

all_lm = [(2,2),(2,1),(3,3),(3,2),(3,1),(4,4),(4,3),(4,2),(4,1),(5,5),(6,6),(7,7),(8,8)]
zero_at_equal_mass_ratio = [(2,1),(3,3),(3,1),(4,3),(4,1),(5,5),(7,7)]

PN_dom_func = {}

for lm in all_lm:
    if lm in zero_at_equal_mass_ratio:
        PN_dom_func[lm] = lambda eta: eta*np.sqrt(1 - 4*eta)
    else:
        PN_dom_func[lm] = lambda eta: eta

In [72]:
mode_string_pro_list = ['2.2.0', '-2.2.0', '2.2.1', '2.1.0', '-2.1.0', '2.1.1', '3.3.0', '3.3.1', '3.2.0', '4.4.0', '5.5.0']

for mode_string_pro in mode_string_pro_list:
    l, m = abs(int(mode_string_pro.split('.')[0])), int(mode_string_pro.split('.')[1])
    mode_string_plot = mode_string_pro.replace('.', ',')
    mode_string_plot_tex = mode_string_pro.replace('.', r'{,}')
    if mode_string_plot[0] == '-':
        mode_string_plot = '𝑟' + mode_string_plot[1:]
        mode_string_plot_tex = r'r' + mode_string_plot_tex[1:]
        PN_func = PN_dom_func[(l,m)]
    else:
        PN_func = PN_dom_func[(l,m)]

    # screen = [1110]
    if mode_string_pro == '5.5.0':
        screen = [110, 40]
    else:
        screen = []
    df_mode_3D = df_get_mode_3D_full(df, l, m, mode_string_pro, SXS_screen = screen, chi_low = 0.1, include_retro = False)
    df_mode_3D = df_mode_3D.dropna()
    # df_mode_3D = df_mode_3D[df_mode_3D['SXS_num'] > 305]
    df_mode_3D['A_med_adj'] = df_mode_3D['A_med'] / PN_func(df_mode_3D['eta'])
    df_mode_3D['A_med_log'] = np.log10(df_mode_3D['A_med'])
    df_mode_3D['A_comp'] = df_mode_3D['A_med_adj'] #* np.exp(1j*df_mode_3D['phi_med_adj'])
    df_mode_3D['A_comp_unadj'] = df_mode_3D['A_med'] #* np.exp(1j*df_mode_3D['phi_med_adj'])
    df_mode_3D['A_real'] = np.real(df_mode_3D['A_comp'])
    df_mode_3D['A_imag'] = np.imag(df_mode_3D['A_comp'])
    df_mode_3D['A_range'] = np.abs(df_mode_3D['A_hi'] - df_mode_3D['A_low'])/df_mode_3D['A_med']
    df_mode_3D['dA'] = np.abs(df_mode_3D['A_hi'] - df_mode_3D['A_low'])/2
    df_mode_3D['A_range_inv'] = 1/df_mode_3D['A_range']
    df_mode_3D['A_range_adj_inv'] = PN_func(df_mode_3D['eta'])*df_mode_3D['A_range_inv']
    df_mode_3D['A_range_rel_inv'] = df_mode_3D['A_med']*df_mode_3D['A_range_inv']
    df_mode_3D['phi_range'] = np.abs(df_mode_3D['phi_hi'] - df_mode_3D['phi_low'])
    df_mode_3D['dphi'] = np.abs(df_mode_3D['phi_hi'] - df_mode_3D['phi_low'])/2
    df_mode_3D['phi_range_inv'] = 1/df_mode_3D['phi_range']
    df_mode_3D['A_range_adj_abs'] = np.sqrt((np.abs(df_mode_3D['A_hi'] - df_mode_3D['A_low']))**2 + (df_mode_3D['A_med']*(df_mode_3D['phi_hi'] - df_mode_3D['phi_low']))**2) / PN_func(df_mode_3D['eta']) /2
    df_mode_3D['A_range_adj_abs_inv'] = 1/df_mode_3D['A_range_adj_abs']
    df_mode_3D['A_range_adj_abs_rel_inv'] = df_mode_3D['A_med_adj']*df_mode_3D['A_range_adj_abs_inv']
    df_mode_3D['A_range_abs'] = df_mode_3D['A_range_adj_abs'] * PN_func(df_mode_3D['eta'])
    df_mode_3D['A_range_abs_inv'] = 1/df_mode_3D['A_range_abs']
    df_mode_3D_full = df_get_mode_3D_full(df_full, l, m, mode_string_pro, SXS_screen = screen, chi_low = 0.1, include_retro = False)
    df_mode_3D_full = df_mode_3D_full.dropna()
    df_mode_3D_full['A_med_adj'] = df_mode_3D_full['A_med'] / PN_func(df_mode_3D_full['eta'])
    df_mode_3D_full['A_med_log'] = np.log10(df_mode_3D_full['A_med'])
    df_mode_3D_full['A_comp'] = df_mode_3D_full['A_med_adj'] * np.exp(1j*df_mode_3D_full['phi_med_adj'])
    df_mode_3D_full['A_comp_unadj'] = df_mode_3D_full['A_med'] * np.exp(1j*df_mode_3D_full['phi_med_adj'])
    df_mode_3D_full['A_real'] = np.real(df_mode_3D_full['A_comp'])
    df_mode_3D_full['A_imag'] = np.imag(df_mode_3D_full['A_comp'])
    df_mode_3D_full['A_range'] = np.abs(df_mode_3D_full['A_hi'] - df_mode_3D_full['A_low'])/df_mode_3D_full['A_med']
    df_mode_3D_full['dA'] = np.abs(df_mode_3D_full['A_hi'] - df_mode_3D_full['A_low'])/2
    df_mode_3D_full['A_range_inv'] = 1/df_mode_3D_full['A_range']
    df_mode_3D_full['A_range_adj_inv'] = PN_func(df_mode_3D_full['eta'])*df_mode_3D_full['A_range_inv']
    df_mode_3D_full['A_range_rel_inv'] = df_mode_3D_full['A_med']*df_mode_3D_full['A_range_inv']
    df_mode_3D_full['phi_range'] = np.abs(df_mode_3D_full['phi_hi'] - df_mode_3D_full['phi_low'])
    df_mode_3D_full['dphi'] = np.abs(df_mode_3D_full['phi_hi'] - df_mode_3D_full['phi_low'])/2
    df_mode_3D_full['phi_range_inv'] = 1/df_mode_3D_full['phi_range']
    df_mode_3D_full['A_range_adj_abs'] = np.sqrt((np.abs(df_mode_3D_full['A_hi'] - df_mode_3D_full['A_low']))**2 + (df_mode_3D_full['A_med']*(df_mode_3D_full['phi_hi'] - df_mode_3D_full['phi_low']))**2) / PN_func(df_mode_3D_full['eta']) /2
    df_mode_3D_full['A_range_adj_abs_inv'] = 1/df_mode_3D_full['A_range_adj_abs']
    df_mode_3D_full['A_range_adj_abs_rel_inv'] = df_mode_3D_full['A_med_adj']*df_mode_3D_full['A_range_adj_abs_inv']
    df_mode_3D_full['A_range_abs'] = df_mode_3D_full['A_range_adj_abs'] * PN_func(df_mode_3D_full['eta'])
    df_mode_3D_full['A_range_abs_inv'] = 1/df_mode_3D_full['A_range_abs']


    df_mode_3D_full_full = df_mode_3D_full.copy()
    df_mode_3D_full = df_mode_3D_full[
        (~((df_mode_3D_full['SXS_num'] >= 209) & (df_mode_3D_full['SXS_num'] <= 305))) & 
        (~((df_mode_3D_full['SXS_num'] >= 1419) & (df_mode_3D_full['SXS_num'] <= 1509)))]
    equal_mass_ratio_tol = 0.24

    if (l,m) in zero_at_equal_mass_ratio:
        df_mode_3D = df_mode_3D[df_mode_3D['eta'] < equal_mass_ratio_tol]
        df_mode_3D_full = df_mode_3D_full[df_mode_3D_full['eta'] < equal_mass_ratio_tol]
    # if mode_string_pro == '4.4.0':
    #     df_mode_3D = df_mode_3D[df_mode_3D['eta'] < 0.225]
    #     df_mode_3D_full = df_mode_3D_full[df_mode_3D_full['eta'] < 0.225]
    def phase_push(phase, theta):
        return (phase + theta)%(2*np.pi)
    def unwrap_crude(df, eta_cut, theta, thresh = np.pi, up = True, SXS_minus = [], SXS_plus = []):
        phase_pushed = phase_push(df['phi_med_adj'], theta)
        if up:
            phase_pushed[(df['eta'] < eta_cut) & (phase_pushed < thresh)] += 2*np.pi 
            # phase_pushed[(eta > eta_cut) & (phase_pushed > np.pi)] -= 2*np.pi
        else:
            # phase_pushed[(eta > eta_cut) & (phase_pushed < np.pi)] += 2*np.pi 
            phase_pushed[(df['eta'] < eta_cut) & (phase_pushed > thresh)] -= 2*np.pi
        for SXS_num in SXS_minus:
            if SXS_num in df['SXS_num'].values:
                phase_pushed[df['SXS_num'] == SXS_num] -= 2*np.pi
        for SXS_num in SXS_plus:
            if SXS_num in df['SXS_num'].values:
                phase_pushed[df['SXS_num'] == SXS_num] += 2*np.pi
        return phase_pushed - theta
    push_dict = {'3.2.0': (0.2, np.pi/2-0.43, np.pi, False, [1468, 2131, 1459], []),
                '4.4.0': (0., -0.9, np.pi, False, [1122, 156], []),
                '5.5.0': (0., -0.5, np.pi, False, [], []),
                '-2.2.0': (0.17, -2.3, np.pi, True, [], []),
                '-2.1.0': (0.17, -2.8, 1, True, [], [1441, 1426])}
    df_mode_3D_full_full['phase_pushed'] = phase_push(df_mode_3D_full_full['phi_med_adj'], -2.4)
    if mode_string_pro in push_dict.keys():
        eta_cut, theta, thresh, up, SXS_minus, SXS_plus = push_dict[mode_string_pro]
        df_mode_3D_full_full['phase_unwrapped'] = unwrap_crude(df_mode_3D_full_full, eta_cut, theta, thresh, up, SXS_minus, SXS_plus)
        df_mode_3D['phase_unwrapped'] = unwrap_crude(df_mode_3D, eta_cut, theta, thresh, up, SXS_minus, SXS_plus)
        df_mode_3D_full['phase_unwrapped'] = unwrap_crude(df_mode_3D_full, eta_cut, theta, thresh, up, SXS_minus, SXS_plus)
    else:
        df_mode_3D_full_full['phase_unwrapped'] = df_mode_3D_full_full['phi_med_adj']
        df_mode_3D['phase_unwrapped'] = df_mode_3D['phi_med_adj']
        df_mode_3D_full['phase_unwrapped'] = df_mode_3D_full['phi_med_adj']
    # import plotly.express as px
    # fig = px.scatter_3d(df_mode_3D_full_full, x='chi_p', y='chi_m', z='eta', color='phase_unwrapped', opacity=0.5, color_continuous_scale='viridis',
    #                     hover_name = 'SXS_num')
    # # fig = px.scatter_3d(df_mode_3D_full, x='chi_p', y='chi_m', z='eta', color='phase_pushed', opacity=0.5, color_continuous_scale='viridis')
    # fig.show()
    A = df_mode_3D['A_med']
    dA = df_mode_3D['dA']
    phi = df_mode_3D['phase_unwrapped']
    dphi = df_mode_3D['dphi']
    eta = df_mode_3D['eta']
    chi_p = df_mode_3D['chi_p']
    chi_m = df_mode_3D['chi_m']
    q = df_mode_3D['q']
    chi_1 = df_mode_3D['chi_1_z']
    chi_2 = df_mode_3D['chi_2_z']

    interpolate_data_dict = {'A': A, 'dA': dA, 'phi': phi, 'dphi': dphi, 'eta': eta, 'chi_p': chi_p, 'chi_m': chi_m, 'q': q, 'chi_1': chi_1, 'chi_2': chi_2}
    for key in interpolate_data_dict.keys():
        interpolate_data_dict[key] = interpolate_data_dict[key].to_list()

    with open(f"../../pickle/interpolate_data/{mode_string_pro}_interpolate_data.json", "w") as f:
        json.dump(interpolate_data_dict, f)

    points_eta_chi_p_chi_m = np.array([eta, chi_p, chi_m]).T

1468
2131
1459
1468
1459
2131
1122
156
1122
156


In [89]:
with open(f"../../pickle/interpolate_data/5.5.0_interpolate_data.json", "r") as f:
    interpolator_data_dict = json.load(f)

In [90]:
import jaxqualin.hyper
from jaxqualin.hyper import *

In [98]:
reload(jaxqualin.hyper)
from jaxqualin.hyper import *

In [104]:
interpolator_dict = make_hyper_interpolator(interpolate_data_dict, PN = False)

In [105]:
A_interp = interpolator_dict['A']([2, 0.1, -0.1])
dA_interp = interpolator_dict['dA']([2, 0.1, -0.1])
phi_interp = interpolator_dict['phi']([2, 0.1, -0.1])
dphi_interp = interpolator_dict['dphi']([2, 0.1, -0.1])

In [106]:
print(A_interp, dA_interp, phi_interp, dphi_interp)

[0.0139963] [9.41465187e-05] [2.78888697] [0.0084431]
